# Automated Ebay web scraping and price monitoring Script with automated email  notification. 

In [29]:
# Importing the necessary libraries
from bs4 import BeautifulSoup
import requests
import time 
import datetime 
import pandas as pd 
import csv 
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [11]:
# connect to the website
url = 'https://www.ebay.com/itm/266676977002?itmmeta=01HQ2SR4Z9VPCHYES1SZR71683&hash=item3e172fbd6a:g:TXoAAOSwWM5lzrhP&itmprp=enc%3AAQAIAAAA4CxpUeU%2FcWJH6%2BgKDuMkydTQ%2Fdq7Y3t1%2FFkJA3Nsx4a40MEexQQT3lsNs2uP4Ati3wRvih9CaqKqxrV4PVA6cpT%2BXTp39LajQv8mg8therALoyTXT176mkkv06%2BiLZUbZmRM2cF%2FfYOfMLVpx49GZEyYV3Qdwu3tOWHzBwmw6BwEGEzaj41paK8b8psNrRn8yWLWDUciBLxuTNkEFj0l%2Bu7PHEo9kPT%2BaroSGEIPC9DvsRoFmQZQAFkeo%2Bn%2B0gFVKZbgJML9svx071nPs6xRHmm2z5EqjDzrZpOURuy01i2q%7Ctkp%3ABk9SR_zP4Nm4Yw'
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42","Accept-Encoding": "gzip, deflate","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9","Upgrade-Insecure-Requests": "1"}
page = requests.get(url, headers)

# Create a BeautifulSoup object
soup1 = BeautifulSoup(page.content, 'html.parser')
soup2 = BeautifulSoup(soup1.prettify(), "html.parser") # TO MAKE IT MORE READABLE
# check the content of the page
# print(soup2)

# Extract the title of the product
div = soup2.find('div', {"class":"vim x-item-title"})
title = div.get_text()
print(title)

# Extract the price of the product
div = soup2.find('div', {"class":"x-price-primary"})
price = div.get_text()
print(price)









              Dell Latitude 5410 Laptop Core i5-10310U 2.21GHz 4GB 128GB SSD 14" Win11 PRO
             










                US $149.00
               






### cleaning the Data 

In [12]:
# We need to clean the data to make it more readable

price = price.strip()[4:]
title = title.strip()
print(price)
print(title)
# now are our data is clean. This process can be done for multiple data entries.

149.00
Dell Latitude 5410 Laptop Core i5-10310U 2.21GHz 4GB 128GB SSD 14" Win11 PRO


In [13]:
# now we add the date 

today = datetime.date.today()
print(today)

2024-02-20


### Writing data to a csv file 

In [14]:
#  now we need tp create the CSV file where will store our data
header = ['Title', 'Price','Date']
data = [title,price,today]

with open('EbayWebScraping.csv', 'w',newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

### Reading data from a CSV 

In [17]:
# Now we need to open the CSV and check if the data is stored
df = pd.read_csv('C:/Users/muson/Documents/internship/projects/Web-Scraping/EbayWebScraping.csv')
print(df)

                                               Title  Price        Date
0  Dell Latitude 5410 Laptop Core i5-10310U 2.21G...  149.0  2024-02-20


# Function to send email

In [43]:
#  The following code will send and email to the user if the price of the product is below a certain value
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

def send_email():
    # Email configuration
    sender_email = os.getenv('SENDER_EMAIL')
    receiver_email = os.getenv('RECEIVER_EMAIL')
    password = os.getenv('SENDER_PASSWORD')

    # Create message
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = 'Price Reduction for Dell Laptop!'

    body = 'Hello, The Price for the Dell laptop you have been watching has reduce to '+ str(price)
    message.attach(MIMEText(body, 'plain'))

    # Connect to the SMTP server
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)
        text = message.as_string()
        server.sendmail(sender_email, receiver_email, text)
        print('Email sent successfully!')
    except Exception as e:
        print(f'Error: {e}')
    finally:
        server.quit()
send_email()

Email sent successfully!


# Now we automate the Process 

In [40]:
"""Now we create a function that will monitor the price of 
the product and store the data in the CSV file"""

def monitor_price():
    # connect to the website
    url = 'https://www.ebay.com/itm/266676977002?itmmeta=01HQ2SR4Z9VPCHYES1SZR71683&hash=item3e172fbd6a:g:TXoAAOSwWM5lzrhP&itmprp=enc%3AAQAIAAAA4CxpUeU%2FcWJH6%2BgKDuMkydTQ%2Fdq7Y3t1%2FFkJA3Nsx4a40MEexQQT3lsNs2uP4Ati3wRvih9CaqKqxrV4PVA6cpT%2BXTp39LajQv8mg8therALoyTXT176mkkv06%2BiLZUbZmRM2cF%2FfYOfMLVpx49GZEyYV3Qdwu3tOWHzBwmw6BwEGEzaj41paK8b8psNrRn8yWLWDUciBLxuTNkEFj0l%2Bu7PHEo9kPT%2BaroSGEIPC9DvsRoFmQZQAFkeo%2Bn%2B0gFVKZbgJML9svx071nPs6xRHmm2z5EqjDzrZpOURuy01i2q%7Ctkp%3ABk9SR_zP4Nm4Yw'
    headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42","Accept-Encoding": "gzip, deflate","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9","Upgrade-Insecure-Requests": "1"}
    page = requests.get(url, headers)

    # Create a BeautifulSoup object
    # Create a BeautifulSoup object
    soup1 = BeautifulSoup(page.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser") # TO MAKE IT MORE READABLE

    # Extract the title of the product
    div = soup2.find('div', {"class":"vim x-item-title"})
    title = div.get_text()

    # Extract the price of the product
    div = soup2.find('div', {"class":"x-price-primary"})
    price = div.get_text()

    # Formating the data now    
    price = price.strip()[4:]
    title = title.strip()
    today = datetime.date.today() #adding the date field 
    
    # We create the fields for the csv 
    header = ['Title', 'Price','Date']
    data = [title,price,today]
    with open('EbayWebScraping.csv', 'a+',newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

    if float(price) < 149.00:
        send_email()

# We will run the function every 24 hours
while(True):
    monitor_price()
    time.sleep(86400) # 24 hours in seconds


Email sent successfully!
